Import libraries

In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

Extract data from Zillow.com

- price
- type of real estate
- sqft
- number of bedrooms
- number of bathrooms
- geographic location
- postal code

Data Scraping from Zillow

In [101]:
soup = BeautifulSoup(content,'lxml')

# setup zillow info
header = {
    'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding':'gzip, deflate',
    'accept-language':'en-US,en;q=0.9',
    'upgrade-insecure-requests':'1',
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    #'referer':'https://www.zillow.com/coquitlam-bc/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-123.22050242009584%2C%22east%22%3A-122.59908823552553%2C%22south%22%3A49.07533567746665%2C%22north%22%3A49.31987107874593%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A791378%2C%22regionType%22%3A6%7D%2C%7B%22regionId%22%3A791534%2C%22regionType%22%3A6%7D%2C%7B%22regionId%22%3A791535%2C%22regionType%22%3A6%7D%2C%7B%22regionId%22%3A791533%2C%22regionType%22%3A6%7D%2C%7B%22regionId%22%3A791530%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D'
}
url = 'https://www.zillow.com/metro-vancouver-regional-district-bc/1_p/'
url2 = 'https://www.zillow.com/metro-vancouver-regional-district-bc/1_p/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-126.49231698339844%2C%22east%22%3A-121.52100350683594%2C%22south%22%3A48.385297573767375%2C%22north%22%3A50.45561673409384%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A404635%2C%22regionType%22%3A4%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A9%7D'
# get the response
response = requests.get(url = url2 , headers = header)
#response.status_code
# status_code is 200 = good


In [133]:
# Initalize soup
soup = BeautifulSoup(response.content,'html.parser')
#print(soup.prettify()) # okay it works

In [ ]:
# Test to see if we can grab correct information
listings = soup.find('li',class_='ListItem-c11n-8-100-4__sc-13rwu5a-0 StyledListCardWrapper-srp-8-100-4__sc-wtsrtn-0 ggnGBj iXHkOx')
#print(listings.prettify())

# sale price
pricing = listings.find('div',class_='PropertyCardWrapper__StyledPriceGridContainer-srp-8-100-4__sc-16e8gqd-0 eCcCyo').text
#print(pricing)

# beds / baths / sqft
bds = listings.find('ul',class_='StyledPropertyCardHomeDetailsList-c11n-8-100-4__sc-1j0som5-0 dPkwmL').text
#print(bds)

# type of unit (house/appartment/conda/etc)
# note this actually gets bd/ba/sqft/unit-type (will need to clean this after)
unit_type = listings.find('div',class_='StyledPropertyCardDataArea-c11n-8-100-4__sc-10i1r6-0 loHGQM').text
#print(unit_type)

# mortage broker
broker = listings.find('div',class_='StyledPropertyCardDataArea-c11n-8-100-4__sc-10i1r6-0 dRseUs').text
#print(broker)

# address
location = listings.find('address').text
#print(location)

# long and lat
data_geo = [
    json.loads(x.string) for x in listings.find("script", type="application/ld+json")
]
for d in data_geo:
    geographic = d["geo"]
    new = list(geographic.values())
    lat = new[2]
    long = new[3]
  

In [ ]:
# Test if we have collected the correct information
for house in listings:
    pricing = listings.find('div',class_='PropertyCardWrapper__StyledPriceGridContainer-srp-8-100-4__sc-16e8gqd-0 eCcCyo').text
    # note this actually gets bd/ba/sqft/unit-type (will need to clean this after)
    detail = listings.find('div',class_='StyledPropertyCardDataArea-c11n-8-100-4__sc-10i1r6-0 loHGQM').text
    # we also want to find mortage broker
    broker = listings.find('div',class_='StyledPropertyCardDataArea-c11n-8-100-4__sc-10i1r6-0 dRseUs').text
    print(broker)
    #address
    address = listings.find('address').text
    
    data_geo = [json.loads(x.string) for x in listings.find("script", type="application/ld+json")]
    for d in data_geo:
        geographic = d["geo"]
        temp = list(geographic.values())
        lat = temp[2]
        long = temp[3]
  
    # check if we get all the info
    print(f'''
    Price: {pricing}
    Details: {detail}
    Broker_info: {broker}
    Address: {address}
    Long: {long}
    lat: {lat}
    ''')

Iterate through all pages from Zillow (Greater Vancouver)

In [ ]:
sales_price = []
detail_list = []
broker_list = []
address_list = []

for page in range(1,21):
    response = requests.get(url = f'https://www.zillow.com/metro-vancouver-regional-district-bc/{page}_p/' , headers = header)
    soup = BeautifulSoup(response.content,'html.parser')
    for price in soup.find_all('div',class_='PropertyCardWrapper__StyledPriceGridContainer-srp-8-100-4__sc-16e8gqd-0 eCcCyo'):
        temp_prices = price.span.text
        sales_price.append(temp_prices)
    for details in soup.find_all('div',class_='StyledPropertyCardDataArea-c11n-8-100-4__sc-10i1r6-0 loHGQM'):
        temp = details.text
        detail_list.append(temp)
    for broker in soup.find_all('div',class_='StyledPropertyCardDataArea-c11n-8-100-4__sc-10i1r6-0 dRseUs'):
        temp = broker.text
        broker_list.append(temp)
    for address in soup.find_all('a',class_='StyledPropertyCardDataArea-c11n-8-100-4__sc-10i1r6-0 dKKiyn property-card-link'):
        temp = address.text
        address_list.append(temp)

Create dataframe to store all information

In [ ]:
sales_df = pd.DataFrame({'SalesPrice':sales_price,'Housing_Details':detail_list,'Broker':broker_list,'address':address_list})

Export to csv

In [ ]:
sales_df.to_csv('Zillow_Housing_data.csv')